In [1]:
function calculate_branching(y)
    v = 0
    found = false
    for m in 1:nv(pool)

# UNCOMMENT FOR BRANCHING
         if modf(y[m])[1] >ϵ  # Check for fractional variables
             v = m # TO COMPLETE
             found = true
             break
         end
# UNCOMMENT FOR BRANCHING             
        found && break
    end
    return v
end

calculate_branching (generic function with 1 method)

In [ ]:
function solve_BP()
    global column_pool = Array{Array{Int,1},1}()
    # this is a 2-dimensional array
    # column_pool[i] is a pattern  (1-dimensional array = vector)
    # pattern vectors are  such that:
    #   -the first element is the objective value contribution
    #   -the second to n+1 st elements are 1 or 0 depending on whether nth pair is included in the pattern or not


    push!(column_pool,vcat(1,ones(Int,nv(pool)))) # push an artificial pattern 
    push!(column_pool,zeros(Int,nv(pool)+1))    # push an empty pattern 


    # set the global lower and upper bound to +,- infinity
    global UB = Inf  # If we have a heuristic solution its value can be used   
    global LB = -Inf # If we have an initial relaxation bound it can be used 

    global Queue = Vector{Int}() # the Queue of nodes to process is an array of integers
    global tree = Vector{TreeNode}() # the branch and price tree is an array of TreeNode objects
    
    # the branch and price tree is initialized with the root node
    # the root node is its own parent, it has no children (at the moment), its LB is -infinity, 
    # there are no variables set to 0 or 1 at the root node
    push!(tree,TreeNode(0,[],Inf,[],[]))   
    push!(Queue,1)  # the indices of nodes to be processed are pushed to the Queue

    while length(Queue)>0
        println("Current list of nodes to be processed:", Queue)
        # we will process the last node in the Queue
        current = Queue[end]
        # Process_Node is the function that solves the restricted master LP
        # for the current node and returns its optimal solution
        # If the master LP is infeasible then returns []
        y = Process_Node(current) # relaxation par generation de colonne
        

# UNCOMMENT FOR BRANCHING        
         # if the current node is not infeasible and its upper
         # bound is promising then we branch
         if size(y,1)!=0 && tree[current].ub >= LB
             # in order to find a variable x_ij to branch on
             # we call the function calculate_branching()
             # it currently returns the first pair of (machine,job) indices
             # such that x_ij fractional
             # if no such pair is found then it returns 0
             v = calculate_branching(y)
            
             # if we have found a variable to branch on 
             # then we proceed with adding the children node
             # otherwise the optimal master LP solution is integer
             # there is no need for branching
             if v!=0
                 println("Two new nodes are created branching on variable x[$job,$machine]")
                 push!(tree,TreeNode(current,[],tree[current].ub,vcat(v,tree[current].setzero),tree[current].setone))
                 push!(tree[current].children,length(tree))
                 push!(Queue,length(tree))
                 push!(tree,TreeNode(current,[],tree[current].ub,tree[current].setzero,vcat(v,tree[current].setone)))
                 push!(tree[current].children,length(tree))
                 push!(Queue,length(tree))
             else
                 println("The optimal relaxation solution was integer")
                 println("Feasible solution with value $(tree[current].ub) found")
             end
         else
             println("The node is either infeasible or pruned by bound")
         end
# UNCOMMENT FOR BRANCHING        
        
        # calculate the global upper bound as the
        # maximum of upperer bounds among all currently active nodes
        global UB = tree[current].ub
        for i in Queue
            if tree[i].ub >= UB global UB=tree[i].ub end
        end
        # print the current global lower and upper bounds
        println("LB=$LB,UB=$UB")
        # remove the processed node from the Queue
        deleteat!(Queue,findfirst(x -> x == current, Queue))
        # remove any nodes whose bound is not promising from the Queue
        deleteat!(Queue,unique(nodestobedeleted))
        # STOP if the optimality gap is sufficiently small
        if abs(2*(UB-LB)/(UB+LB))<=ϵ
            break
        end
    end
end